In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import condorgmm
import condorgmm.eval.metrics
import condorgmm.data
from tqdm import tqdm
import fire
import datetime as dt
import importlib
import condorgmm.camera_tracking.integrated_camera_tracking
import condorgmm.camera_tracking.em_camera_tracking
import warp as wp
import matplotlib.pyplot as plt

In [ ]:
initialize = condorgmm.camera_tracking.integrated_camera_tracking.initialize
update = condorgmm.camera_tracking.integrated_camera_tracking.update
rr_log = condorgmm.camera_tracking.integrated_camera_tracking.rr_log

In [ ]:
original_video = condorgmm.data.TUMVideo("freiburg1_desk2")
video = original_video.downscale(4)
plt.imshow(video[0].rgb / 255)
plt.imshow(video[0].depth)

In [ ]:
import condorgmm.warp_gmm as warp_gmm
import importlib
importlib.reload(condorgmm.warp_gmm.optimize)
importlib.reload(condorgmm.warp_gmm)

condorgmm.rr_init("tum_05")

camera_pose, st, meta0 = initialize(video[0])
rr_log(st, video[0], timestep=0)

inferred_pose, st, meta1 = update(st, video[0], timestep=1, do_matter_update_override=False, debug=True)
rr_log(st, video[0], timestep=1, do_log_frame=True, do_log_condor_state=True)

In [ ]:
frame = video[1]
# frame.rgb = frame.rgb + np.random.randn(*frame.rgb.shape)
# frame.depth = frame.depth + np.random.randn(*frame.depth.shape) * 0.01

inferred_pose, st, meta1 = update(st, frame, timestep=2, do_matter_update_override=False, debug=True)
rr_log(st, frame, timestep=2, do_log_frame=True, do_log_condor_state=True)

In [ ]:
inference_log_score_image = st.warp_gmm_state.log_score_image.numpy()

print(inference_log_score_image.sum())

plt.matshow(inference_log_score_image)


In [ ]:
wst = st.warp_gmm_state
wst.gmm.camera_posquat = wp.array(frame.camera_pose.astype(np.float32))
frame_warp = frame.as_warp()
warp_gmm.warp_gmm_forward(frame_warp, wst)
gt_log_score_image = wst.log_score_image.numpy()
print(gt_log_score_image.sum())
plt.matshow(gt_log_score_image)

In [ ]:
plt.matshow(inference_log_score_image - gt_log_score_image, cmap='bwr')
plt.colorbar()

In [ ]:
plt.imshow(video[1].rgb / 255)


In [ ]:
condorgmm.rr_log_frame(
    video[0], camera_pose=video[0].camera_pose, channel="GT-FRAME-0"
)
condorgmm.rr_log_frame(
    video[1], camera_pose=video[1].camera_pose, channel="GT-FRAME-1"
)
condorgmm.rr_log_frame(
    video[4], camera_pose=video[4].camera_pose, channel="GT-FRAME-4"
)

In [ ]:
import condorgmm.warp_gmm as warp_gmm
import importlib
importlib.reload(condorgmm.warp_gmm.optimize)
importlib.reload(condorgmm.warp_gmm)

condorgmm.rr_init("tum_06")

camera_pose, st, meta0 = initialize(video[0])
rr_log(st, video[0], timestep=0)

inferred_pose, st, meta1 = update(st, video[0], timestep=1, do_matter_update_override=False, debug=True)
rr_log(st, video[0], timestep=1, do_log_frame=True, do_log_condor_state=True)

for t in tqdm(range(1, 12)):
    camera_pose, st, meta = update(st, video[t-1], timestep=t, do_matter_update_override=False, debug=True)
    rr_log(st, video[t-1], timestep=t, do_log_frame=True, do_log_condor_state=True)

In [ ]:
condorgmm.rr_init("tum_04")

In [ ]:
import condorgmm.warp_gmm as warp_gmm
import importlib
importlib.reload(condorgmm.warp_gmm.optimize)
importlib.reload(condorgmm.warp_gmm)

camera_pose, st, meta0 = initialize(video[0])
rr_log(st, video[0], timestep=0, do_log_frame=True, do_log_condor_state=True)

print("Initialized.")

log = [meta0]
for t in tqdm(range(1, 5)):
    camera_pose, st, meta = update(st, video[t], timestep=t, do_matter_update_override=False, debug=True)
    rr_log(st, video[t], timestep=t, do_log_frame=True, do_log_condor_state=True)
    log.append(meta)

In [ ]:
plt.plot(log[1]['optimization_data']['likelihoods'][80:])
# print(log[3]['optimization_data']['likelihoods'][-5:])

In [ ]:
import numpy as np
print(np.max(log[1]['optimization_data']['likelihoods'][400:]))

In [ ]:
import rerun as rr

for (t, posquat) in enumerate(log[3]['optimization_data']['params']):
    rr.set_time_sequence("ADAM2", t)
    condorgmm.rr_log_posquat(posquat[0])
    condorgmm.rr_log_posquat(video[4].camera_pose, channel="gt")

In [ ]:
# t = 1
# camera_pose, st, meta = update(st, video[t], timestep=t, do_matter_update_override=False, debug=True)

In [ ]:
import condorgmm.warp_gmm as warp_gmm
import importlib
importlib.reload(condorgmm.warp_gmm.optimize)
importlib.reload(condorgmm.warp_gmm)


warp_gmm.warp_gmcamera_pose, st, _ = initialize(video[0])
warp_gmm.warp_gmm_forward(video[0].as_warp(), st.warp_gmm_state)
print(st.warp_gmm_state.log_score_image.numpy().sum())
plt.matshow(st.warp_gmm_state.log_score_image.numpy())

t = 0
import condorgmm.warp_gmm as warp_gmm
learning_rates = wp.array(
    [0.002, 0.002, 0.002, 0.0004, 0.0004, 0.0004, 0.0004], dtype=wp.float32
)
frame = video[t]
frame_warp = frame.as_warp()
st.warp_gmm_state.gmm.camera_posquat.requires_grad = True
optimization_data = warp_gmm.optimize_params(
    [st.warp_gmm_state.gmm.camera_posquat],
    frame_warp,
    st.warp_gmm_state,
    num_timesteps=4,
    lr=learning_rates,
    storing_stuff=True,
)
print("after")
print(st.warp_gmm_state.log_score_image.numpy().sum())
warp_gmm.warp_gmm_forward(frame_warp, st.warp_gmm_state)
print(st.warp_gmm_state.log_score_image.numpy().sum())
print(optimization_data["likelihoods"][-10:])
print(optimization_data["params"][-0:])

In [ ]:
plt.matshow(warp_gmm_state.log_score_image.numpy())

In [ ]:
log = []
for t in tqdm(range(1, 5)):
    camera_pose, st, meta = update(st, video[t], timestep=t, do_matter_update_override=False, debug=True)
    rr_log(st, video[t], timestep=t, do_log_frame=True, do_log_condor_state=True)
    log.append(meta)

In [ ]:
plt.plot(log[3]['optimization_data']['likelihoods'][-5:])
print(log[3]['optimization_data']['likelihoods'][-5:])

In [ ]:
import condorgmm.warp_gmm as warp_gmm
warp_gmm.warp_gmm_forward(video[4].as_warp(), st.warp_gmm_state)
st.warp_gmm_state.log_score_image.numpy().sum()



In [ ]:
import rerun as rr

for (t, posquat) in enumerate(log[1]['optimization_data']['params']):
    rr.set_time_sequence("ADAM2", t)
    condorgmm.rr_log_posquat(posquat[0])
    condorgmm.rr_log_posquat(video[1].camera_pose, channel="gt")

In [ ]:
log[3]['optimization_data']['params'][-1]

In [ ]:
import numpy as np
# st.warp_gmm_state.gmm.camera_posquat = wp.array(np.array(video[4].camera_pose, dtype=np.float32))
st.warp_gmm_state.gmm.camera_posquat = wp.array(np.array(log[3]['optimization_data']['params'][-1][0], dtype=np.float32))

In [ ]:
# gt:  348548.4
# inf: 321617.47

In [ ]:
condorgmm.utils.common.rerun.rr_log_frame(
    video[4], "observed_data_v2", camera_pose=condorgmm.Pose(video[4].camera_pose)
)